# Authorization and Imports


In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
%load_ext google.colab.data_table

In [5]:
%%bigquery --project plucky-zodiac-283501 df
SELECT *
FROM cleandata.combineddata

In [6]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [50]:
import pandas as pd

In [7]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tokenizing").getOrCreate()

# Read in Data and Place in Dataframe

In [8]:
# Read in data from storage
from pyspark import SparkFiles

url ="https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/1975-1979.csv"
url2 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/1980-1989.csv"
url3 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/1990-2001.csv"
url4 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/2002-2009.csv"
url5 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/2010-2015.csv"
url6 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/2016_avg.csv"

In [9]:
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("1975-1979.csv"), sep=",", header=True)

In [10]:
spark.sparkContext.addFile(url2)
df2 = spark.read.csv(SparkFiles.get("1980-1989.csv"), sep=",", header=True)

In [11]:
spark.sparkContext.addFile(url3)
df3 = spark.read.csv(SparkFiles.get("1990-2001.csv"), sep=",", header=True)

In [12]:
spark.sparkContext.addFile(url4)
df4 = spark.read.csv(SparkFiles.get("2002-2009.csv"), sep=",", header=True)

In [13]:
spark.sparkContext.addFile(url5)
df5 = spark.read.csv(SparkFiles.get("2010-2015.csv"), sep=",", header=True)

In [14]:
spark.sparkContext.addFile(url6)
df6 = spark.read.csv(SparkFiles.get("2016_avg.csv"), sep=",", header=True)

In [15]:
dfUnion = df.union(df2)

In [16]:
dfUnion2 = dfUnion.union(df3)

In [17]:
dfUnion3 = dfUnion.union(df4)

In [18]:
dfUnion4 = dfUnion.union(df5)

In [19]:
dfUnion5 = dfUnion.union(df6)

In [20]:
dfUnion5.show()

+----+-----+-------+--------------------+------+--------+-------+-------------------+------------------+------------------+------------------+------------------+
|year|state|station|                name|   lat|     lon|   elev|           avg_temp|           avg_dew|           avg_slp|           avg_stp|           avg_co2|
+----+-----+-------+--------------------+------+--------+-------+-------------------+------------------+------------------+------------------+------------------+
|1975|   NH| 726160|BERLIN MUNICIPAL ...|44.583| -71.179|+0353.0| 48.552121212121207|40.676515151515147|5781.6575757575747|2755.7356060606062|329.85166666666669|
|1975|   NY| 725060|   OTIS ANGB AIRPORT| 41.65| -70.061|+0047.5| 54.339075268817204|45.689440860215058|            9999.9|1012.6343655913978|331.20300000000003|
|1975|   OR| 726885|             MEACHAM|45.511|-118.425|+1134.8| 40.082917735608305|3928.3748064282372|9102.0103225806452|  9087.61322580645|           331.521|
|1975|   AK| 700265|BARROW W

# Machine Learning: Train, Test, Fit and Summarize

In [21]:
from pyspark.ml.feature import VectorAssembler

In [22]:
df_modified = dfUnion5.drop('name', 'lon', 'avg_slp', 'avg_stp', 'state', 'station', 'avg_dew')

# station = six-digit number that is a string, no decimals. 

df_modified.show()

+----+------+-------+-------------------+------------------+
|year|   lat|   elev|           avg_temp|           avg_co2|
+----+------+-------+-------------------+------------------+
|1975|44.583|+0353.0| 48.552121212121207|329.85166666666669|
|1975| 41.65|+0047.5| 54.339075268817204|331.20300000000003|
|1975|45.511|+1134.8| 40.082917735608305|           331.521|
|1975|71.333|+0004.0|               15.0|            328.34|
|1975|70.233|-0999.0| 4.5260755682066796|           332.224|
|1975|65.617|+0005.0| 14.700000000000001|            331.65|
|1975|69.367|+0085.0|-12.867118298368297|            329.53|
|1975|67.511|+0585.0|-10.939230769230772|           330.125|
|1975|  63.9|+0214.0|  28.33787867996201|331.28999999999996|
|1975|  62.1|+0566.3| 27.595451631701632|           331.521|
|1975|61.977|+0045.7| 33.186844007707975|332.17249999999996|
|1975| 64.55|+0110.0| 8.2347069597069584|            329.51|
|1975|63.033|+0823.0| 24.320588235294121|331.40818181818184|
|1975|  62.1|+0573.0| 41

In [23]:
df_modified = df_modified.selectExpr("cast(year as int) year",
    "cast(lat as double) lat",
    "cast(elev as double) elev", 
    "cast(avg_temp as double) avg_temp", 
    "cast(avg_co2 as double) avg_co2")

df_modified.printSchema()

root
 |-- year: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- elev: double (nullable = true)
 |-- avg_temp: double (nullable = true)
 |-- avg_co2: double (nullable = true)



In [24]:
features = df_modified.drop("avg_temp")

In [25]:
assembler = VectorAssembler(inputCols=features.columns, outputCol="features", handleInvalid="skip")

In [26]:
output = assembler.transform(df_modified).select('features','avg_temp')

In [27]:
train,test = output.randomSplit([0.75, 0.25], seed=42) 

In [28]:
train.show()

+--------------------+------------------+
|            features|          avg_temp|
+--------------------+------------------+
|[1975.0,18.255,10...| 79.11708141321044|
|[1975.0,18.336,6....| 81.09575010118418|
|[1975.0,18.498,66...| 79.40661411822988|
|[1975.0,19.719,11...| 72.35902585765488|
|[1975.0,19.76,188...|58.107898479152425|
|[1975.0,20.79,399...| 71.07767868014932|
|[1975.0,21.317,15...| 75.18506464413724|
|[1975.0,21.324,2....| 75.23113799283155|
|[1975.0,21.45,7.3...| 75.23592165898617|
|[1975.0,21.984,30...| 75.04279185867894|
|[1975.0,22.033,7....| 75.93634641896564|
|[1975.0,24.557,0....| 79.18689260112649|
|[1975.0,24.583,1....| 78.99674859190989|
|[1975.0,24.633,1....| 77.21010822510823|
|[1975.0,24.95,1.0...| 79.06531364468864|
|[1975.0,25.483,2....| 75.74278673835124|
|[1975.0,25.788,8....| 76.77397593445983|
|[1975.0,26.072,3....| 78.24742962286138|
|[1975.0,26.184,30...| 73.21689132104456|
|[1975.0,26.228,11...| 77.92969779314889|
+--------------------+------------

In [29]:
test.show()

+--------------------+-----------------+
|            features|         avg_temp|
+--------------------+-----------------+
|[1975.0,18.433,2....|79.68193484383002|
|[1975.0,20.001,81...|       66.9203125|
|[1975.0,20.9,15.5...|74.15759408602152|
|[1975.0,21.155,13...|76.06389411853509|
|[1975.0,21.487,25...|72.82809042675284|
|[1975.0,24.72,1.0...|77.56029761904762|
|[1975.0,25.914,7....|73.02472862263184|
|[1975.0,27.072,5....|             68.9|
|[1975.0,27.401,9....|74.34494815668202|
|[1975.0,27.5,15.2...|70.66938300051203|
|[1975.0,27.6,1.0,...|            73.15|
|[1975.0,27.617,16...|77.99364197530863|
|[1975.0,27.85,4.3...|73.53958781362006|
|[1975.0,28.101,11...|75.30717933947774|
|[1975.0,28.233,3....|76.31795285359802|
|[1975.0,28.367,20...|68.96269393241167|
|[1975.0,28.434,27...|71.91521761392728|
|[1975.0,28.483,3....|72.41583589349719|
|[1975.0,28.95,2.0...|67.92679346515554|
|[1975.0,29.378,31...|68.41613927291348|
+--------------------+-----------------+
only showing top

In [30]:
from pyspark.ml.regression import LinearRegression
lin_reg = LinearRegression(featuresCol = 'features', labelCol='avg_temp')
linear_model = lin_reg.fit(train)
linear_model2 = lin_reg.fit(test)

In [31]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("\nr2: %f" % trainSummary.r2)

RMSE: 6.153825

r2: 0.755504


Root MSE: absolute measure of fit. Standard deviation of unexplained variance / square root of the variance of the residuals. 

R-squared: Relative measure of fit. Approximately three-quarters of the observed variation can be explained by the model's inputs. 

In [32]:
testSummary = linear_model2.summary
print("RMSE: %f" % testSummary.rootMeanSquaredError)
print("\nr2: %f" % testSummary.r2) # 74.5202% 

RMSE: 6.130369

r2: 0.760389


Root MSE: absolute measure of fit. standard deviation of unexplained variance / square root of the variance of the residuals. 

R-squared: relative measure of fit. approximately three-quarters of the observed variation can be explained by the model's inputs.

# Make Predictions Using Trained Model


In [68]:
url7 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/US_Capitols_new.csv"

In [69]:
spark.sparkContext.addFile(url7)
df7 = spark.read.csv(SparkFiles.get("US_Capitols_new.csv"), sep=",", header=True)

In [70]:
df8 = df7[["Year","State", "City"]]

df8.show()

+----+-----+------------+
|Year|State|        City|
+----+-----+------------+
|2025|   AL|  Montgomery|
|2025|   AK|      Juneau|
|2025|   AZ|     Phoenix|
|2025|   AR| Little Rock|
|2025|   CA|  Sacramento|
|2025|   CO|      Denver|
|2025|   CT|    Hartford|
|2025|   DE|       Dover|
|2025|   FL| Tallahassee|
|2025|   GA|     Atlanta|
|2025|   HI|    Honolulu|
|2025|   ID|       Boise|
|2025|   IL| Springfield|
|2025|   IN|Indianapolis|
|2025|   IA|  Des Moines|
|2025|   KS|      Topeka|
|2025|   KY|   Frankfort|
|2025|   LA| Baton Rouge|
|2025|   ME|     Augusta|
|2025|   MD|   Annapolis|
+----+-----+------------+
only showing top 20 rows



In [72]:
df7 = df7.drop('State', 'City')

In [73]:
df_modified2 = df7.drop('Elevation_ft', 'Dew_Point')

df_modified2.show()

+----+-------+-----------+------+
|Year|    Lat|Elevation_m|   CO2|
+----+-------+-----------+------+
|2025|   32.3|     73.152|424.49|
|2025|  58.37|    17.0688|424.49|
|2025|  33.43|   331.0128|424.49|
|2025|  35.22|    102.108|424.49|
|2025|  38.52|      9.144|424.49|
|2025|  39.75|   1609.344|424.49|
|2025|  41.73|    17.9832|424.49|
|2025|  39.13|      9.144|424.49|
|2025|  30.38|    61.8744|424.49|
|2025|  33.65|     320.04|424.49|
|2025|21.3294|   6.001512|424.49|
|2025|43.6007|    832.104|424.49|
|2025|  39.85|   170.0784|424.49|
|2025|39.7771|    217.932|424.49|
|2025|  41.53|    291.084|424.49|
|2025|  39.07|    288.036|424.49|
|2025|38.1924|   155.1432|424.49|
|2025|  30.53|    17.0688|424.49|
|2025|  44.32|    20.7264|424.49|
|2025|  38.97|    11.8872|424.49|
+----+-------+-----------+------+
only showing top 20 rows



In [74]:
df_modified2 = df_modified2.selectExpr("cast(Year as int) Year",
    "cast(Lat as double) Lat",
    "cast(Elevation_m as double) Elevation_m", 
    "cast(CO2 as double) CO2")

In [75]:
features2 = df_modified2

features2.show()

+----+-------+-----------+------+
|Year|    Lat|Elevation_m|   CO2|
+----+-------+-----------+------+
|2025|   32.3|     73.152|424.49|
|2025|  58.37|    17.0688|424.49|
|2025|  33.43|   331.0128|424.49|
|2025|  35.22|    102.108|424.49|
|2025|  38.52|      9.144|424.49|
|2025|  39.75|   1609.344|424.49|
|2025|  41.73|    17.9832|424.49|
|2025|  39.13|      9.144|424.49|
|2025|  30.38|    61.8744|424.49|
|2025|  33.65|     320.04|424.49|
|2025|21.3294|   6.001512|424.49|
|2025|43.6007|    832.104|424.49|
|2025|  39.85|   170.0784|424.49|
|2025|39.7771|    217.932|424.49|
|2025|  41.53|    291.084|424.49|
|2025|  39.07|    288.036|424.49|
|2025|38.1924|   155.1432|424.49|
|2025|  30.53|    17.0688|424.49|
|2025|  44.32|    20.7264|424.49|
|2025|  38.97|    11.8872|424.49|
+----+-------+-----------+------+
only showing top 20 rows



In [76]:
assembler2 = VectorAssembler(inputCols=features2.columns, outputCol="features", handleInvalid="skip")

In [77]:
output2 = assembler2.transform(features2).select('features')

output2.show()

+--------------------+
|            features|
+--------------------+
|[2025.0,32.3,73.1...|
|[2025.0,58.37,17....|
|[2025.0,33.43,331...|
|[2025.0,35.22,102...|
|[2025.0,38.52,9.1...|
|[2025.0,39.75,160...|
|[2025.0,41.73,17....|
|[2025.0,39.13,9.1...|
|[2025.0,30.38,61....|
|[2025.0,33.65,320...|
|[2025.0,21.3294,6...|
|[2025.0,43.6007,8...|
|[2025.0,39.85,170...|
|[2025.0,39.7771,2...|
|[2025.0,41.53,291...|
|[2025.0,39.07,288...|
|[2025.0,38.1924,1...|
|[2025.0,30.53,17....|
|[2025.0,44.32,20....|
|[2025.0,38.97,11....|
+--------------------+
only showing top 20 rows



In [78]:
lin_reg2 = LinearRegression(featuresCol = 'features')

In [79]:
lr_predictions = linear_model.transform(output2)

In [80]:
lr_predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[2025.0,32.3,73.1...| 66.08284438364342|
|[2025.0,58.37,17....| 35.83139041698871|
|[2025.0,33.43,331...|63.603375320117834|
|[2025.0,35.22,102...| 62.53634445711816|
|[2025.0,38.52,9.1...|59.092419652673584|
|[2025.0,39.75,160...| 50.47134496141314|
|[2025.0,41.73,17....| 55.29689260948123|
|[2025.0,39.13,9.1...| 58.37869010052043|
|[2025.0,30.38,61....| 68.37995280193115|
|[2025.0,33.65,320...|63.395212079809994|
|[2025.0,21.3294,6...| 79.22035838438262|
|[2025.0,43.6007,8...| 49.45419817388108|
|[2025.0,39.85,170...|56.813959757554244|
|[2025.0,39.7771,2...|56.684482828118576|
|[2025.0,41.53,291...| 54.30518878788632|
|[2025.0,39.07,288...| 57.19718780371238|
|[2025.0,38.1924,1...| 58.82046326952002|
|[2025.0,30.53,17....| 68.40553915787802|
|[2025.0,44.32,20....|52.254155279484735|
|[2025.0,38.97,11....| 58.55358599744352|
+--------------------+------------

In [81]:
lr_pandas = lr_predictions.toPandas()

lr_pandas.head()

,features,prediction
0,"[2025.0, 32.3, 73.152, 424.49]",66.082844
1,"[2025.0, 58.37, 17.0688, 424.49]",35.831390
2,"[2025.0, 33.43, 331.0128, 424.49]",63.603375
3,"[2025.0, 35.22, 102.108, 424.49]",62.536344
4,"[2025.0, 38.52, 9.144, 424.49]",59.092420


In [82]:
df8_pandas = df8.toPandas()

df8_pandas.head()

,Year,State,City
0,2025,AL,Montgomery
1,2025,AK,Juneau
2,2025,AZ,Phoenix
3,2025,AR,Little Rock
4,2025,CA,Sacramento


In [83]:
combined_df = pd.concat([df8_pandas, lr_pandas], axis=1)

In [84]:
combined_df.head()

,Year,State,City,features,prediction
0,2025,AL,Montgomery,"[2025.0, 32.3, 73.152, 424.49]",66.082844
1,2025,AK,Juneau,"[2025.0, 58.37, 17.0688, 424.49]",35.831390
2,2025,AZ,Phoenix,"[2025.0, 33.43, 331.0128, 424.49]",63.603375
3,2025,AR,Little Rock,"[2025.0, 35.22, 102.108, 424.49]",62.536344
4,2025,CA,Sacramento,"[2025.0, 38.52, 9.144, 424.49]",59.092420


In [85]:
final_predictions = combined_df.drop(['features'], axis=1)

In [86]:
final_predictions.head()

,Year,State,City,prediction
0,2025,AL,Montgomery,66.082844
1,2025,AK,Juneau,35.831390
2,2025,AZ,Phoenix,63.603375
3,2025,AR,Little Rock,62.536344
4,2025,CA,Sacramento,59.092420


In [89]:
final_predictions.to_csv('capital_predictions.csv', index=False) 